## Experiments on the ACAS Xu benchmarks

The networks (in .nnet format) and properties are taken from https://github.com/guykatzz/ReluplexCav2017/tree/master/nnet 
(also available at onnx format with properties at https://github.com/stanleybak/vnncomp2021/tree/main/benchmarks/acasxu) 

Convert in Julia file by jupyter nbconvert --to script ACASXu.ipynb

In [1]:
include("../src/Pbox_nn.jl")
include("../src/Sets_utils.jl")
include("../src/DSZ.jl")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


paf_focal_refinement_sauv (generic function with 1 method)

## Specifying input ranges, networks and properties

In [2]:
# from https://github.com/stanleybak/vnncomp2021/blob/main/benchmarks/acasxu/generate.py

init_lb_prop_1_2 = [55947.691, -pi, -pi, 1145, 0]
init_ub_prop_1_2 = [60760, pi, pi, 1200, 60]
acas_input_1_2 = interval.(init_lb_prop_1_2,init_ub_prop_1_2)

init_lb_prop_3 = [1500, -0.06, 3.1, 980, 960]
init_ub_prop_3 = [1800, 0.06, pi, 1200, 1200]
acas_input_3 = interval.(init_lb_prop_3,init_ub_prop_3)
    
init_lb_prop_4 = [1500, -0.06, 0, 1000, 700]
init_ub_prop_4 = [1800, 0.06, 0, 1200, 800]
acas_input_4 = interval.(init_lb_prop_4,init_ub_prop_4)

init_lb_prop_5 = [250, 0.2, -3.141592, 100, 0]
init_ub_prop_5 = [400, 0.4, -3.141592 + 0.005, 400, 400]
acas_input_5 = interval.(init_lb_prop_5,init_ub_prop_5)

init_lb_prop_61 = [12000, 0.7, -3.141592, 100, 0]
init_ub_prop_61 = [62000, 3.141592, -3.141592 + 0.005, 1200, 1200]
acas_input_61 = interval.(init_lb_prop_61,init_ub_prop_61)

init_lb_prop_62 = [12000, -3.141592, -3.141592, 100, 0]
init_ub_prop_62 = [62000, -0.7, -3.141592 + 0.005, 1200, 1200]
acas_input_62 = interval.(init_lb_prop_62,init_ub_prop_62)

init_lb_prop_7 = [0, -3.141592, -3.141592, 100, 0]
init_ub_prop_7 = [60760, 3.141592, 3.141592, 1200, 1200]
acas_input_7 = interval.(init_lb_prop_7,init_ub_prop_7)

init_lb_prop_8 = [0, -3.141592, -0.1, 600, 600]
init_ub_prop_8 = [60760, -0.75*3.141592, 0.1, 1200, 1200]
acas_input_8 = interval.(init_lb_prop_8,init_ub_prop_8)

init_lb_prop_9 = [2000, -0.4, -3.141592, 100, 0]
init_ub_prop_9 = [7000, -0.14, -3.141592 + 0.01, 150, 150]
acas_input_9 = interval.(init_lb_prop_9,init_ub_prop_9)

#output labels = ['Clear of Conflict (COC)', 'Weak Left', 'Weak Right', 'Strong Left', 'Strong Right']


function get_spec(prop::Int64)
    if (prop == 2)
        desc = "Unsafe if COC is maximal"
        # Unsafe if y1 > y2 and y1 > y3 and y1 > y4 and y1 > y5
        mat = [[-1. 1. 0. 0. 0.]
               [-1. 0. 1. 0. 0.]
               [-1. 0. 0. 1. 0.]
               [-1. 0. 0. 0. 1.]]
        rhs = [0., 0., 0., 0.]
    elseif (prop == 3) || (prop == 4)
        desc = "Unsafe if COC is minimal"
        mat = [[1. -1. 0. 0. 0.]
               [1. 0. -1. 0. 0.]
               [1. 0. 0. -1. 0.]
               [1. 0. 0. 0. -1.]]
        rhs = [0., 0., 0., 0.]
    end

    return (desc, mat, rhs)
end

mat_spec_2 = get_spec(2)[2]
rhs_spec_2 = get_spec(2)[3]
mat_spec_3_4 = get_spec(3)[2]
rhs_spec_3_4 = get_spec(3)[3]

mat_essai_1 = [[-1. 1. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]]

mat_essai_2 = [[0. 0. 0. 0. 0.]
[-1. 0. 1. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]]

mat_essai_3 = [[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[-1. 0. 0. 1. 0.]
[0. 0. 0. 0. 0.]]

mat_essai_4 = [[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[-1. 0. 0. 0. 1.]]


acas_nnet_1_2 = read_nnet("./ACASXU_networks/ACASXU_run2a_1_2_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_1_3 = read_nnet("./ACASXU_networks/ACASXU_run2a_1_3_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_1_4 = read_nnet("./ACASXU_networks/ACASXU_run2a_1_4_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_1_5 = read_nnet("./ACASXU_networks/ACASXU_run2a_1_5_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_1_6 = read_nnet("./ACASXU_networks/ACASXU_run2a_1_6_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_2_2 = read_nnet("./ACASXU_networks/ACASXU_run2a_2_2_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_2_9 = read_nnet("./ACASXU_networks/ACASXU_run2a_2_9_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_3_1 = read_nnet("./ACASXU_networks/ACASXU_run2a_3_1_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_3_6 = read_nnet("./ACASXU_networks/ACASXU_run2a_3_6_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_3_7 = read_nnet("./ACASXU_networks/ACASXU_run2a_3_7_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_4_1 = read_nnet("./ACASXU_networks/ACASXU_run2a_4_1_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_4_7 = read_nnet("./ACASXU_networks/ACASXU_run2a_4_7_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_5_3 = read_nnet("./ACASXU_networks/ACASXU_run2a_5_3_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_1_7 = read_nnet("./ACASXU_networks/ACASXU_run2a_1_7_batch_2000.nnet", last_layer_activation = Id());
acas_nnet_1_9 = read_nnet("./ACASXU_networks/ACASXU_run2a_1_9_batch_2000.nnet", last_layer_activation = Id());


# Prop 2: x0 >= 0.6
# x0 <= 0.6798577687
# x1 >= -0.5
# x1 <= 0.5
# x2 >= -0.5
# x2 <= 0.5
# x3 >= 0.45
# x3 <= 0.5
# x4 >= -0.5
# x4 <= -0.45
# +y0 -y1 >= 0
# +y0 -y2 >= 0
# +y0 -y3 >= 0
# +y0 -y4 >= 0

init_lb_prop_1_2 = [0.6, -0.5, -0.5, 0.45, -0.5]
init_ub_prop_1_2 = [0.6798577687, 0.5, 0.5, 0.5, -0.45]
acas_input_1_2 = interval.(init_lb_prop_1_2,init_ub_prop_1_2)

# Prop 3:
# x0 >= -0.3035311561
# x0 <= -0.2985528119
# x1 >= -0.0095492966
# x1 <= 0.0095492966
# x2 >= 0.4933803236
# x2 <= 0.5
# x3 >= 0.3
# x3 <= 0.5
# x4 >= 0.3
# x4 <= 0.5
# +y0 -y1 <= 0
# +y0 -y2 <= 0
# +y0 -y3 <= 0
# +y0 -y4 <= 0

init_lb_prop_3 = [-0.3035311561, -0.0095492966, 0.4933803236, 0.3, 0.3]
init_ub_prop_3 = [-0.2985528119, 0.0095492966, 0.5, 0.5, 0.5]
acas_input_3 = interval.(init_lb_prop_3,init_ub_prop_3)

# Prop4:
# x0 >= -0.3035311561
# x0 <= -0.2985528119
# x1 >= -0.0095492966
# x1 <= 0.0095492966
# x2 >= 0
# x2 <= 0
# x3 >= 0.3181818182
# x3 <= 0.5
# x4 >= 0.0833333333
# x4 <= 0.1666666667
# +y0 -y1 <= 0
# +y0 -y2 <= 0
# +y0 -y3 <= 0
# +y0 -y4 <= 0

init_lb_prop_4 = [-0.3035311561, -0.0095492966, 0.0, 0.3181818182, 0.0833333333]
init_ub_prop_4 = [-0.2985528119, 0.0095492966,0.0, 0.5, 0.1666666667]
acas_input_4 = interval.(init_lb_prop_4,init_ub_prop_4)


5-element Vector{Interval{Float64}}:
    [-0.303532, -0.298552]
    [-0.0095493, 0.0095493]
 [0, 0]
     [0.318181, 0.5]
     [0.0833333, 0.166667]

## DSZ Analysis

### Property 2

In [3]:
# Defining the number of focal element for each component of the input vector for Property 2
vect_nb_focal_elem = [5, 80, 50, 6, 5]
println("vect_nb_focal_elem for Property 2 = ",vect_nb_focal_elem,":")
# the true flag in init_pbox_Normal means truncating the focal elements to restrict the range to [lb,ub]
acas_inputpbox_1_2 = init_pbox_Normal(init_lb_prop_1_2,init_ub_prop_1_2,vect_nb_focal_elem,true)

vect_nb_focal_elem for Property 2 = [5, 80, 50, 6, 5]:


5-element Vector{pbox}:
 Pbox: 	  ~ normal ( range=[0.6, 0.67986], mean=0.63993, var=0.00017715)
 Pbox: 	  ~ normal ( range=[-0.5, 0.5], mean=0.0, var=0.027778)
 Pbox: 	  ~ normal ( range=[-0.5, 0.5], mean=0.0, var=0.027778)
 Pbox: 	  ~ normal ( range=[0.45, 0.5], mean=0.475, var=6.9444e-5)
 Pbox: 	  ~ normal ( range=[-0.5, -0.45], mean=-0.475, var=6.9444e-5)

In [4]:
println("Property 2, net-1-6 :")
@time dsz_approximate_nnet_and_condition_nostorage(acas_nnet_1_6, acas_inputpbox_1_2, mat_spec_2,rhs_spec_2) 

println("Property 2, net-2-2 :")
@time dsz_approximate_nnet_and_condition_nostorage(acas_nnet_2_2, acas_inputpbox_1_2, mat_spec_2,rhs_spec_2)

println("Property 2, net-2-9 :")
@time dsz_approximate_nnet_and_condition_nostorage(acas_nnet_2_9, acas_inputpbox_1_2, mat_spec_2,rhs_spec_2) 

println("Property 2, net-3-1 :")
@time dsz_approximate_nnet_and_condition_nostorage(acas_nnet_3_1, acas_inputpbox_1_2, mat_spec_2,rhs_spec_2) 

println("Property 2, net-3-6 :")
@time dsz_approximate_nnet_and_condition_nostorage(acas_nnet_3_6, acas_inputpbox_1_2, mat_spec_2,rhs_spec_2) 

println("Property 2, net-3-7 :")
@time dsz_approximate_nnet_and_condition_nostorage(acas_nnet_3_7, acas_inputpbox_1_2, mat_spec_2,rhs_spec_2)

println("Property 2, net-4-1 :")
@time dsz_approximate_nnet_and_condition_nostorage(acas_nnet_4_1, acas_inputpbox_1_2, mat_spec_2,rhs_spec_2)

println("Property 2, net-4-7 :")
@time dsz_approximate_nnet_and_condition_nostorage(acas_nnet_4_7, acas_inputpbox_1_2, mat_spec_2,rhs_spec_2)

println("Property 2, net-5-3 :")
@time dsz_approximate_nnet_and_condition_nostorage(acas_nnet_5_3, acas_inputpbox_1_2, mat_spec_2,rhs_spec_2)

Property 2, net-1-6 :
Number of zonotopic focal elements=600000
Probability : [0, 0.0199884]
 49.622368 seconds (98.81 M allocations: 53.059 GiB, 26.57% gc time, 2.90% compilation time)
Property 2, net-2-2 :
Number of zonotopic focal elements=600000
Probability : [0.00423499, 0.0808967]
 49.326549 seconds (94.72 M allocations: 55.679 GiB, 27.12% gc time)
Property 2, net-2-9 :
Number of zonotopic focal elements=600000
Probability : [0, 0.0774684]
 54.966083 seconds (95.30 M allocations: 59.746 GiB, 27.26% gc time)
Property 2, net-3-1 :
Number of zonotopic focal elements=600000
Probability : [0.0164666, 0.0878734]
 44.923243 seconds (94.54 M allocations: 48.860 GiB, 27.33% gc time)
Property 2, net-3-6 :
Number of zonotopic focal elements=600000
Probability : [0.0167383, 0.111137]
 54.313320 seconds (97.13 M allocations: 59.857 GiB, 26.85% gc time)
Property 2, net-3-7 :
Number of zonotopic focal elements=600000
Probability : [5.99999e-05, 0.136119]
 46.511809 seconds (96.97 M allocations:

[0, 0.0393934]

### Properties 3 and 4

In [5]:
# Defining the number of focal element for each component of the input vector for Properties 3 and 4
vect_nb_focal_elem = [5, 20, 1, 6, 5]
println("vect_nb_focal_elem for Properties 3 and 4 = ",vect_nb_focal_elem,":")
acas_inputpbox_3 = init_pbox_Normal(init_lb_prop_3,init_ub_prop_3,vect_nb_focal_elem,true)

println("Property 3, net-1-7 :")
@time dsz_approximate_nnet_and_condition_nostorage(acas_nnet_1_7, acas_inputpbox_3, mat_spec_3_4,rhs_spec_3_4) 

vect_nb_focal_elem for Properties 3 and 4 = [5, 20, 1, 6, 5]:
Property 3, net-1-7 :
Number of zonotopic focal elements=3000
Probability : [1, 1]
  0.227095 seconds (488.67 k allocations: 292.097 MiB, 28.29% gc time)


[1, 1]

In [6]:
#vect_nb_focal_elem = [5, 20, 1, 6, 5] 
acas_inputpbox_4 = init_pbox_Normal(init_lb_prop_4,init_ub_prop_4,vect_nb_focal_elem,true)

println("Property 4, net-1-9 :")
@time dsz_approximate_nnet_and_condition_nostorage(acas_nnet_1_9, acas_inputpbox_4, mat_spec_3_4,rhs_spec_3_4) 

Property 4, net-1-9 :
Number of zonotopic focal elements=3000
Probability : [1, 1]
  0.201004 seconds (467.16 k allocations: 196.915 MiB, 36.25% gc time)


[1, 1]